In [101]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import sqlalchemy
import pytz
import psycopg2

In [57]:
campaign_offers = dwh_query("egypt", "select * from materialized_views.Campaign_logs", "read")
campaign_offers

/home/ec2-user/service_account_key.json


,campaign_id,objective,supplier_id,offer_type,active_base,churnedr_base,churnedo_base,comp_base,total_budget,sending_budget,offers_budget,consumed_budget,start_at,end_at,campaign_name
0,1,increase_nmv,171,sku_discounts,500,600,200,1000,70000,10000,60000.0,1000.0,2025-06-11 13:00:00,2025-07-01 10:00:00,None
1,2,increase_base,723,brand_specific_coupon,520,590,210,1010,65000,9000,56000.0,2000.0,2025-06-12 09:00:00,2025-07-02 10:00:00,None
2,3,increase_nmv,929,offers,510,620,205,980,72000,12000,60000.0,1500.0,2025-06-13 14:30:00,2025-07-03 11:30:00,None
3,4,increase_base,928,offers,530,610,215,1020,68000,9500,58500.0,2500.0,2025-06-14 12:00:00,2025-07-04 09:45:00,None
4,5,increase_nmv,750,quantity_discount,540,605,220,990,75000,11000,64000.0,1800.0,2025-06-15 10:00:00,2025-07-05 10:15:00,None
5,6,increase_nmv,1948,offers,560,615,225,995,73000,10500,62500.0,2300.0,2025-06-16 08:00:00,2025-07-06 11:00:00,None
6,7,increase_base,728,brand_specific_coupon,570,625,230,1015,67000,9500,57500.0,1900.0,2025-06-17 15:00:00,2025-07-07 14:00:00,None
7,8,increase_nmv,651,sku_discounts,580,635,240,1030,74000,10000,64000.0,2600.0,2025-06-18 16:00:00,2025-07-08 12:00:00,None
8,9,increase_base,267,offers,590,645,250,1040,71000,9800,61200.0,2100.0,2025-06-19 11:00:00,2025-07-09 09:00:00,None
9,10,increase_nmv,267,quantity_discount,600,655,260,1060,76000,11500,64500.0,3000.0,2025-06-20 13:00:00,2025-07-10 13:00:00,None


In [58]:
import pandas as pd
campaign_offers = campaign_offers[campaign_offers['offer_type'] == 'offers']
# Make sure end_at is a datetime column
campaign_offers['end_at'] = pd.to_datetime(campaign_offers['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
campaign_ongoing = campaign_offers[campaign_offers['end_at'] > now]
campaign_ongoing


/tmp/ipykernel_5519/860946311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_offers['end_at'] = pd.to_datetime(campaign_offers['end_at'])


,campaign_id,objective,supplier_id,offer_type,active_base,churnedr_base,churnedo_base,comp_base,total_budget,sending_budget,offers_budget,consumed_budget,start_at,end_at,campaign_name
34,34,Increase_NMV,36,offers,91,0,0,0,1000,182,801.0,NaN,2025-10-21 10:50:28,2025-10-28 10:50:28,itteration testing


In [59]:
campaign_ids = campaign_ongoing['campaign_id'].tolist()
print(campaign_ids)


[34]


In [8]:
offers_id = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
offers_id

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [13]:
offers_filtered = offers_id[offers_id['campaign_id'].isin(campaign_ids)]
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-24 12:50:27


In [21]:
import pandas as pd

# Example: updating the 'end_at' value for a specific row
offers_filtered.loc[offers_filtered['id'] == 26, 'end_at'] = pd.Timestamp('2025-10-21 7:17:27')
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-21 07:17:27


In [22]:
offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
offers_ended = offers_filtered[offers_filtered['end_at'] < now]
offers_ended

/tmp/ipykernel_5519/215201034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-21 07:17:27


In [113]:
# Extract the values from offers_ended DataFrame first
if not offers_ended.empty:
    campaign_id_value = offers_ended['campaign_id'].iloc[0]
    supplier_id_value = int(offers_ended['supplier_id'].iloc[0])
    retailer_id_value = offers_ended['retailer_list'].iloc[0]
 
    retailer_list = tuple(retailer_id_value)
    print(f"campaign ID: {campaign_id_value}")
    print(f"Supplier ID: {supplier_id_value}")
    # print(f"Retailer ID: {retailer_id_value}")

else:
    print("offers_ended is empty!")


(669974, 114999, 20102, 179817, 319941, 438891, 152504, 20378, 98617, 184557, 328030, 401755, 608384, 591725, 234212, 190141, 897730, 164698, 158267, 196132, 296222, 635153, 151822, 234304, 521814, 9826, 489128, 214581, 113023, 360562, 380649, 869843, 288673, 132183, 732092, 943802, 504998, 871927, 152598, 238268, 527392, 341763, 134122, 364174, 412875, 391234, 230104, 342497, 411050, 506812, 928694, 152802, 23553, 547912, 9163, 124851, 534321, 220809, 269134, 599447, 332365, 355001, 748009, 326155, 35850, 944592, 301342, 490644, 748947, 199911, 80719, 115532, 38110, 453555, 768536, 294757, 111929, 666819, 155827, 939906, 499538, 25330, 235893, 720849, 680107, 567032, 213854, 378211, 536939, 26641, 208087)
campaign ID: 34
Supplier ID: 36
Retailer ID: [669974, 114999, 20102, 179817, 319941, 438891, 152504, 20378, 98617, 184557, 328030, 401755, 608384, 591725, 234212, 190141, 897730, 164698, 158267, 196132, 296222, 635153, 151822, 234304, 521814, 9826, 489128, 214581, 113023, 360562, 380

In [54]:
query1 = """with offer_redeemers as (
select 	retailer_id,
    	maxab_offer_id,
    	count (distinct parent_sales_order_id) as po_count,
    	sum (discount) as redeemed_value, 
		max(discount) as discount_value
from public.sales_orders so 
where sales_order_status_id in (6,9) 
and maxab_offer_id is not null 
and discount > 0 
and date_trunc(month, so.created_at::date) >= date_trunc ('month',current_date) - interval '6 month'
group by all),

sup_products as (select  distinct sps.parent_supplier_id as supplier_id, 
		ps.product_id
from supplier_parent_suppliers sps
left join product_supplier ps on ps.supplier_id = sps.supplier_id 
left join products p on p.id = ps.product_id 
left join categories c on c.id = p.category_id ),

offer as(
select  s.*, 
		po_count as redeemed_orders, 
		redeemed_value, 
		discount_value
from (
select s.*, 
		TICKET_SIZE_MINIMUM_VALUE
from (
select 	
		SPLIT_PART(description, '_', 2)      AS campaign_id,  
    	REGEXP_SUBSTR(description, '\\d+', 1, 2)  AS supplier_id, 
		offer.id as offer_id, 
		offer.active_from as start_at,
		offer.active_to as end_at, 
    	retailer.value::int as retailer_id, 
		rule_id, 
		description,
		dense_rank() over(partition by campaign_id order by start_at asc) offer_number,
		case when description like '%ch_r%' then 'Churned Recent'
		when description like '%act_c%' then 'Active Competition' 
		when description like '%act%' then 'Active'
		when description like '%ch_o%' then 'Churned Old' end as retailer_tag
from offers.offer, 
    lateral flatten(strtok_to_array(replace(replace(WHITE_LIST_RETAILER_IDS,'{', ''),'}',''),',')) retailer
where start_at::date between current_date - 90 and current_date"""
query2 = f""" and campaign_id = '{campaign_id_value}'
and is_cancelled = false) s 
left join offers.rule r on r.parent_rule_id = s.rule_id and r.dtype = 'TicketSizeRule' )s
left join offer_redeemers ofr on ofr.retailer_id = s.retailer_id and s.offer_id = ofr.maxab_offer_id 
order by s.retailer_id, start_At)

select campaign_id, 
		supplier_id,
        start_at, 
		end_at,
		-- count(distinct product_id) as products_in_campaign, 
		-- retailer_tag, 
		count(distinct retailer_id) as number_retailers, 
	    count(distinct start_at) as number_offers, 
	    count(distinct case when product_nmv_post > 0 then retailer_id end) as converted_retailers, 
	    count(distinct case when product_nmv_lm > 0 then retailer_id end) as converted_prior, 
	   	sum(product_nmv_post) as product_nmv, 
	   	sum(product_nmv_lm) as product_nmv_prior, 
	   	sum(redeemed_value) as total_burn, 
	   	converted_retailers/number_retailers as conversion_rate, 
	   	converted_prior/number_retailers as conversion_rate_pre
from (
select  campaign_id,
		supplier_id, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) start_at, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) end_at,
		offer_number, 
		s.retailer_id, 
		retailer_tag, 
		redeemed_orders, 
		redeemed_value, 
		discount_value, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm, 
		sum(case when p.id is not null and so.created_at between start_at and end_at then coalesce(pso.total_price, 0) else 0 end) as product_nmv_post, 
		sum(case when p.id is not null and so.created_at between start_lm and end_lm then coalesce(pso.total_price, 0) else 0 end) as product_nmv_lm, 
		s.type_id as brand_id 
from (
select  offer.*, 
		type_id, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm 
from offer 
left join (
select distinct rule.parent_rule_id, coalesce(allb.type_id,oneb.type_id) as type_id,ticket_size_minimum_value as off_ts, oneb.minimum_gmv as min_gmv
from offers.rule
left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule') s on s.parent_rule_id = offer.rule_id 
where type_id is not null 
order by retailer_id ) s 
left join sales_orders so on so.retailer_id = s.retailer_id and (CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at) between CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) and CONVERT_TIMEZONE('UTC', 'Africa/Cairo',end_at) or CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at)  between start_lm and end_lm)
left join product_sales_order pso on pso.sales_order_id = so.id and purchased_item_count > 0 
left join products p on p.id = pso.product_id and p.brand_id = s.type_id 
group by all 
order by discount_value ) 

group by all  """
performance = snowflake_query("egypt", f"""
{query1}{query2}
""")
performance

/home/ec2-user/service_account_key.json


,campaign_id,supplier_id,start_at,end_at,number_retailers,number_offers,converted_retailers,converted_prior,product_nmv,product_nmv_prior,total_burn,conversion_rate,conversion_rate_pre
0,34,None,2025-10-21 11:16:00,2025-10-21 11:17:00,2,1,0,0,0.0,0.0,None,0.000000,0.000000
1,34,None,2025-10-21 11:15:00,2025-10-24 11:20:00,2,1,0,0,0.0,0.0,None,0.000000,0.000000


In [63]:
burn = performance['total_burn'].iloc[0] or 0
budget = campaign_ongoing['total_budget'].iloc[0] or 0
budget_left = budget - burn
print(budget_left)


1000


In [67]:
if budget_left > 500 and campaign_ongoing['end_at'].iloc[0] > now:
    print('we good')
else:
    print('Problem')

we good


In [75]:
mid_line1 = "lateral flatten(strtok_to_array(replace(replace(white_list_retailer_ids,'{', ''),'}',''),',')) retailer"
mid_line2 = "lateral flatten(strtok_to_array(replace(replace(retailer_ids,'{', ''),'}',''),',')) retailer"
mid_line3 = "lateral flatten(strtok_to_array(replace(replace(white_list_retailer_ids,'{', ''),'}',''),',')) retailer" 


In [94]:
new_offers = snowflake_query("egypt", f"""
with spbr as (
select distinct
	suppliers.id as supplier_id,
	suppliers.name_en as SUPPLIER_NAME,
	category_id,
	brand_id,
	brands.name_ar as brand_name
from suppliers
	join supplier_parent_suppliers as parent on parent.parent_supplier_id = suppliers.id
	left join purchaseD_orders as po on po.supplier_id = suppliers.id
	left join product_purchased_order as ppo on ppo.purchased_order_id = po.id
	left join products on products.id = ppo.product_id
	left join brands on brands.id = products.brand_id
	left join categories on categories.id = products.category_id
where categories.activation = 'true' and po.created_at >= current_date - interval '6 month' and suppliers.id = {supplier_id_value} 
order by 5 desc,1,3
)

, sizes as (
select 
	*,
	case 
		when max_nmv >= percentile_cont (0.25) within group (order by max_nmv desc) over (partition by 1=1) then 'large'
		when max_nmv >= percentile_cont (0.7) within group (order by max_nmv desc) over (partition by 1=1) then 'med'
		when max_nmv < 50000 then 'minor'
		else 'small'
		end as supplier_size,
	case 
		when supplier_size = 'large' then 0.02
		when supplier_size = 'med' then 0.04
		when supplier_size = 'small' then 0.06
		when supplier_size = 'minor' then 0.08
		end as disc_start,
	case 
		when supplier_size = 'large' then 0.005
		when supplier_size = 'med' then 0.01
		when supplier_size = 'small' then 0.01
		when supplier_size = 'minor' then 0.01
		end as disc_step,
	case 
		when supplier_size = 'large' then 0.04
		when supplier_size = 'med' then 0.06
		when supplier_size = 'small' then 0.08
		when supplier_size = 'minor' then 0.1
		end as disc_cap
from (
select 
	*,
	max (nmv) over (partition by supplier_id) as max_nmv
from (
select 
	time_period,
	supplier_id,
	supplier_name,
	count (distinct retailer_id) as retailers,
	avg (ts) as avg_ts,
	sum (nmv) as nmv
from (
select 
	ceil ((current_date - sales_orders.created_at::Date) / 30 ) as time_period,
	supplier_id,
	supplier_name,
	retailer_id,
	sum (pso.total_price) as nmv,
	avg (pso.total_price) as ts
from product_sales_order as pso 
	left join sales_orders on sales_orders.id = pso.sales_order_id
	left join products on products.id = pso.product_id
	join spbr on spbr.brand_id = products.brand_id 
where pso.created_at >= current_date - interval '90 day' and sales_order_status_id = 6
group by all 
) as sub 
group by all
) as sub2
) as sub3
	where time_period = 1
)

, offer_base as (
select 
	sub.*,
	max(case when sales_orders.id is not null then 1 else 0 end) as ret_redeemed
from (
select distinct 
	offer.id as offer_id,
	campaign_id,
	spbr.supplier_id,
	retailer.value::int as retailer_id,
	split_part(offer.description,'_',2) as extracted_campaign,
	replace(split_part(offer.description,'_',3),'sup','') as extracted_supplier,
	split_part(offer.description,'_',4) as extracted_rt,
	offer.active_from,
	offer.active_to,
	max (ceil ((reward.value*1.0/off_ts)/0.001)*0.001) as disc
from offers.offer 
	left join offers.reward on reward.offer_id = offer.id
	left join 
		(select distinct rule.parent_rule_id, coalesce(allb.type_id,oneb.type_id) as type_id,ticket_size_minimum_value as off_ts from offers.rule
		left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
		left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule'
		) as rule
		on offer.rule_id = rule.parent_rule_id 
	left join spbr  on spbr.brand_id = rule.type_id
	join MATERIALIZED_VIEWS.Campaign_logs as campaigns on campaigns.supplier_id = spbr.supplier_id 
		and offer.active_from between campaigns.start_at and campaigns.end_at and campaigns.offer_type = 'offers'
		and split_part(offer.description,'_',2) = campaigns.campaign_id
	, {mid_line1}
where offer.active_to >= current_date - interval '60 day' and offer.description like 'ct_%' 
	and ((offer.active_to >= CONVERT_TIMEZONE('UTC', 'Africa/Cairo',current_timestamp) and offer.active ='true') or offer.active_to <CONVERT_TIMEZONE('UTC', 'Africa/Cairo',current_timestamp))
	
	and campaigns.supplier_id = {supplier_id_value}
group by all
) as sub 
	left join saleS_orders on sales_orders.retailer_id = sub.retailer_id and sales_orders.maxab_offer_id = sub.offer_id
group by all
)

, top_brands as (
select 
	retailer_id,
	sub3.category_id,
	sub3.brand_id,
	bts,
	bnmv,
	coalesce(case when sup_brand =1 then sub3.brand_id end,spbr.brand_id) sup_brand_id
from (
select 
	*,
	row_number () over (partition by retailer_id,sup_brand order by nmv desc) as cat_rk
from (
select 	
	*,
	max (case when sup_brand=1 then 1 else 0 end) over (partition by retailer_id) as sup_ret
from (
select
	retailer_id,
	products.category_id,
	products.brand_id,
	case when spbr.brand_id is not null then 1 else 0 end as sup_brand,
	sum (pso.total_price) as nmv,
	nmv * 1.0 / count (distinct sales_orders.parent_sales_order_id) as bnmv,
	avg (pso.total_price) as bts
from product_sales_order as pso 
	left join sales_orders on sales_orders.id = pso.sales_order_id
	left join products on products.id = pso.product_id
	join (select distinct supplier_id,category_id from spbr) as cats on cats.category_id = products.category_id
	left join spbr on spbr.category_id = products.category_id and spbr.brand_id = products.brand_id
where sales_orders.created_at >= date_trunc ('month',current_date - interval '3 month')
and retailer_id in {retailer_list}
group by all
) as sub 
) as sub2
) as sub3
	left join spbr on spbr.category_id = sub3.category_id and sup_ret = 0
  where (sup_ret =1 and sup_brand =1 and cat_rk=1) or (sup_ret =0 and cat_rk=1) )

, lastrk as (
select * from (
select 
	*,
	min (days_since_offer) over (partition by retailer_id) as global_days,
	maX(case when supplier_id = {supplier_id_value} then 1 else 0 end) over (partition by retailer_id) as sup_flag,
	row_number () over (partition by retailer_id order by days_since_offer) as support
from (
select 	
	supplier_id,
	retailer_id,
	min (last_offer) as days_since_offer
from (
select distinct 
	spbr.supplier_id,
	value::int as retailer_id,
	current_date - max(sku_discounts.start_at::date) as last_offer
from sku_discounts 
	left join sku_discount_values as val on val.sku_discount_id = sku_discounts.id
	left join products on products.id = val.product_id
	join spbr on spbr.brand_id = products.brand_id
	, {mid_line2}
where sku_discounts.end_At >= current_date - interval '30 day' and sku_discounts.name_en like 'ct_%' 
	and ((sku_discounts.end_At >= CONVERT_TIMEZONE('UTC', 'Africa/Cairo',current_timestamp) and active ='true') or sku_discounts.end_at <CONVERT_TIMEZONE('UTC', 'Africa/Cairo',current_timestamp))
group by all

union all 

select distinct
	coalesce(spbr1.supplier_id,spbr2.supplier_id) as supplier_id,
	value::int as retailer_id,
	current_Date - max(offer.active_from::date) as last_offer
from offers.offer
	JOIN offers.rule on offer.rule_id = rule.parent_rule_id 
	left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
	left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule'
	left join brands on brands.id = oneb.type_id or brands.id = allb.type_id
	left join spbr as spbr1 on spbr1.brand_id = allb.type_id
	left join spbr as spbr2 on spbr2.brand_id = oneb.type_id
	, {mid_line3}
where offer.created_at >= current_date - interval '30 day' and (allb.id is not null or oneb.id is not null) --and spbr.supplier_id::varchar = extracted_supplier 
	and offer.description like 'ct_%'
group by all

union all 

select 
	spbr.supplier_id,
	user_id,
	current_date - max(coupons.created_at::date)
from coupons 
	left join coupons_rules as rules on rules.id = coupons.coupon_rule_id
	join branded_coupon_ui as bc on bc.id = rules.branded_coupon_ui_id and condition_type = 'BRAND'
	left join spbr on spbr.brand_id = bc.conditional_id
	join MATERIALIZED_VIEWS.Campaign_logs as campaigns on campaigns.supplier_id = spbr.supplier_id and coupons.created_at between start_at and end_at 
where coupons.created_at >= current_date - interval '30 day'
group by all
) as sub 
group by all
) as sub2
) as final
	where supplier_id = {supplier_id_value} or (sup_flag =0 and support=1)
)


select * from (
select 
	t2.*,
	rank () over (partition by retailer_type order by coalesce(days_since_offer,31) desc,coalesce(global_Days,31) desc,mobile) as fake_rk
from (
select 
	retailer_id,
	retailer_type,
	supplier_size,
	disc_formatted,
	offer_ts,
	flat_reward,
	case
		when retailer_type = 'active' then 0.5
		when retailer_type in ('churned-recent', 'active-competition') then 0.4
		when retailer_type in ('churned-old') then 0.3
		end as max_redeem_rate,
	case
		when retailer_type = 'active' then 0.3
		when retailer_type in ('churned-recent', 'active-competition') then 0.2
		when retailer_type in ('churned-old') then 0.15
		end as avg_redeem_rate,
	case
		when retailer_type = 'active' then 2
		when retailer_type in ('churned-recent','churned-old') then 5
		when retailer_type in ('active-competition') then 10
		end as sending_cost,
	sum (flat_reward)*max_redeem_rate as max_exp_burn,
	sum (flat_reward)*avg_redeem_rate as avg_exp_burn,
	sum (flat_reward) as max_burn,
	max_exp_burn + sending_cost as total_cost
from (
select 
	lists.*,
	supplier_size,
	1 as new_campaign,
	case 
		when new_campaign = 1 then least(coalesce(disc,disc_start),disc_cap)
		when new_campaign = 0 then least (coalesce(disc,disc_start)+disc_step,disc_cap)
		end as campaign_disc,
	ceil (greatest(case 
		when retailer_type = 'active' and bnmv <= bts*2 then greatest(bnmv,bts * 1.5)
		when retailer_type='active' then bts*1.5
		when retailer_type like 'churned%' then bts
		when retailer_type like 'active-competition' and bts > 1000 then bts * 0.7
		when retailer_type like 'active-competition' then bts
		end,300)/100)*100 as offer_ts,
	greatest(round (campaign_disc * offer_ts),10) as flat_reward,
	bts,
	bnmv,
	concat('[',concat_ws (',',top_brands.sup_brand_id,'1',offer_ts,'50000'),']') as disc_formatted
from (with spbr as 
(
select distinct
	suppliers.id as supplier_id,
	suppliers.name_en as SUPPLIER_NAME,
  	category_id,
	brand_id,
	brands.name_ar as brand_name
from suppliers
	join supplier_parent_suppliers as parent on parent.parent_supplier_id = suppliers.id
	left join purchaseD_orders as po on po.supplier_id = suppliers.id
	left join product_purchased_order as ppo on ppo.purchased_order_id = po.id
	left join products on products.id = ppo.product_id
	left join brands on brands.id = products.brand_id
where po.created_at >= current_date - interval '6 month' 
order by 5 desc,1,3
)

, base as (
select 	
	category_id,
	brand_id,
	retailer_id,
	avg (brand_nmv * 1.0 / orders) as bts,
	avg (brand_nmv) as bnmv,
	current_date - max(ldate) as last_active
from (
select 
	date_trunc ('month',sales_orders.created_at) as month,
	category_id,
	brand_id,
	retailer_id,
	count (distinct parent_sales_order_id) as orders,
	sum (pso.total_price) as brand_nmv,
	max(sales_orders.created_at::date) as ldate
from product_sales_order as pso
	left join sales_orders on sales_orders.id = pso.sales_order_id
	left join products on products.id = pso.product_id
where sales_order_status_id not in (7,12) and sales_orders.created_at >= date_trunc ('month',current_date - interval '3 month')
group by all
) as sub 
group by all
)


, comp as (
select distinct
	base.*,
	categories.name_ar,
	spbr.supplier_id,
	spbr.supplier_name,
	max(case when spbr.brand_id = base.brand_id then 1 else 0 end) over (partition by retailer_id,spbr.supplier_id) as sup_ret,
	avg (base.bts) over (partition by retailer_id,spbr.supplier_id,base.category_id) as bc_ts,
	min (last_active) over (partition by retailer_id,spbr.supplier_id) as min_act
from base
	left join spbr on base.category_id = spbr.category_id
	left join categories on categories.id = base.category_id
order by retailer_id,1,spbr.supplier_id
)	


select 
	*
from (
select 
	spbr.supplier_id,
	spbr.SUPPLIER_NAME,
	retailer_id,
	sum (base.bts) as ts,
	sum (base.bnmv) as nmv,
	min(base.last_active) as la,
	case
		when la<=30 then 'active'
		when la<=75 then 'churned-recent'
		else 'churned-old'
		end as retailer_type
from spbr 
	left join base on spbr.brand_id = base.brand_id and spbr.category_id = base.category_id
group by all

union all 

select 
	spbr.supplier_id,
	spbr.SUPPLIER_NAME,
	retailer_id,
	avg (bc_ts),
	null,
	min(min_act),
	'active-competition'
from spbr
	left join comp on comp.supplier_id = spbr.supplier_id and sup_ret = 0 and min_act <=30
where retailer_id is not null 
group by all 
order by 1,retailer_type 
) 

) as lists
	left join 
		(
		select 
			*,
			case when redeem_perc = max(redeem_perc) over (partition by extracted_rt) then 1 else 0 end as max_redeem,
			case when rank() over (order by start_at::date desc)=1 then 1 else 0 end as last_off
		from (
		select 
			offer_base.campaign_id,
			offer_base.active_from as start_at,
			extracted_rt,
			disc,
			count(distinct case when ret_redeemed=1 then retailer_id end) * 1.0 / count (distinct retailer_id) as redeem_perc
		from offer_base
			group by all
		) as sub
		) as history on history.extracted_rt = lists.retailer_type
	join sizes on sizes.supplier_id = lists.supplier_id
	join top_brands on top_brands.retailer_id = lists.retailer_id
where (
	(new_campaign = 1 and (max_redeem=1 or max_redeem is null)) 
	or (new_campaign = 0 and last_off=1 and lists.retailer_id not in (select retailer_id from offer_base where ret_redeemed=1))
	)
group by all
) as t1
group by all
order by disc_formatted,retailer_type
) as t2 
	left join retailers on retailers.id = t2.retailer_id
	left join lastrk on lastrk.retailer_id = t2.retailer_id
) as final
order by fake_rk,retailer_type


	

	
""")

/home/ec2-user/service_account_key.json


In [92]:
avg_burn = snowflake_query("egypt", f"""
with offer_redeemers as (
select 	retailer_id,
    	maxab_offer_id,
    	count (distinct parent_sales_order_id) as po_count,
    	sum (discount) as redeemed_value, 
		max(discount) as discount_value
from public.sales_orders so 
where sales_order_status_id in (6,9) 
and maxab_offer_id is not null 
and discount > 0 
and date_trunc(month, so.created_at::date) >= date_trunc ('month',current_date) - interval '6 month'
group by all),

sup_products as (select  distinct sps.parent_supplier_id as supplier_id, 
		ps.product_id
from supplier_parent_suppliers sps
left join product_supplier ps on ps.supplier_id = sps.supplier_id 
left join products p on p.id = ps.product_id 
left join categories c on c.id = p.category_id ),

offer as(
select  s.*, 
		po_count as redeemed_orders, 
		redeemed_value, 
		discount_value
from (
select s.*, 
		TICKET_SIZE_MINIMUM_VALUE
from (
select 	
		SPLIT_PART(description, '_', 2)      AS campaign_id,  
    	REGEXP_SUBSTR(description, '\\d+', 1, 2)  AS supplier_id, 
		offer.id as offer_id, 
		offer.active_from as start_at,
		offer.active_to as end_at, 
    	retailer.value::int as retailer_id, 
		rule_id, 
		description,
		dense_rank() over(partition by campaign_id order by start_at asc) offer_number,
		case when description like '%ch_r%' then 'churned-recent'
		when description like '%act_c%' then 'active-competition' 
		when description like '%act%' then 'active'
		when description like '%ch_o%' then 'churned-old' end as retailer_tag
from offers.offer, 
    {mid_line1}
where start_at::date between current_date - 90 and current_date
and is_cancelled = false) s 
left join offers.rule r on r.parent_rule_id = s.rule_id and r.dtype = 'TicketSizeRule' )s
left join offer_redeemers ofr on ofr.retailer_id = s.retailer_id and s.offer_id = ofr.maxab_offer_id 
order by s.retailer_id, start_At)

select 
	q.*,
	active_post - active_pre as active_delta,
	nmv_post - nmv_pre as nmv_delta,
	pct_active_post - pct_active_pre as pct_active_delta
from (
select campaign_id, 
		retailer_tag, 
		count(*) as number_retailers, 
		sum(case when product_nmv_post > 0 then 1 else 0 end) as active_post, 
		sum(case when product_nmv_lm > 0 then 1 else 0 end) active_pre, 
		sum(case when discount_value > 0 then 1 else 0 end) as converted, 
		sum(product_nmv_post) nmv_post, 
		sum(product_nmv_lm) nmv_pre, 
		sum(redeemed_value) as total_burn, 
		sum(case when discount_value > 0 then product_nmv_post else 0 end) as nmv_redeemers, 
		active_post/number_retailers as pct_active_post, 
		active_pre/number_retailers as pct_active_pre
from (
select  campaign_id,
		supplier_id, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) start_at, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) end_at,
		offer_number, 
		s.retailer_id, 
		retailer_tag, 
		redeemed_orders, 
		redeemed_value, 
		discount_value, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm, 
		sum(case when p.id is not null and so.created_at between start_at and end_at then coalesce(pso.total_price, 0) else 0 end) as product_nmv_post, 
		sum(case when p.id is not null and so.created_at between start_lm and end_lm then coalesce(pso.total_price, 0) else 0 end) as product_nmv_lm, 
		s.type_id as brand_id 
from (
select  offer.*, 
		type_id, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm 
from offer 
left join (
select distinct rule.parent_rule_id, coalesce(allb.type_id,oneb.type_id) as type_id,ticket_size_minimum_value as off_ts, oneb.minimum_gmv as min_gmv
from offers.rule
left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule') s on s.parent_rule_id = offer.rule_id 
where type_id is not null 
order by retailer_id ) s 
left join sales_orders so on so.retailer_id = s.retailer_id and (CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at) between CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) and CONVERT_TIMEZONE('UTC', 'Africa/Cairo',end_at) or CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at)  between start_lm and end_lm)
left join product_sales_order pso on pso.sales_order_id = so.id and purchased_item_count > 0 
left join products p on p.id = pso.product_id and p.brand_id = s.type_id 
group by all 
order by discount_value ) 
group by all
) q 
where campaign_id = '{campaign_id_value }'
""")
avg_burn

/home/ec2-user/service_account_key.json


,campaign_id,retailer_tag,number_retailers,active_post,active_pre,converted,nmv_post,nmv_pre,total_burn,nmv_redeemers,pct_active_post,pct_active_pre,active_delta,nmv_delta,pct_active_delta
0,34,active,8,0,0,0,0.0,0.0,None,0.0,0.000000,0.000000,0,0.0,0.000000


In [93]:
import numpy as np

avg_burn['AVERAGE_BURN'] = np.where(
    (avg_burn['number_retailers'] != 0) & (~avg_burn['number_retailers'].isna()),
    avg_burn['total_burn'].fillna(0) / avg_burn['number_retailers'],
    0
)
avg_burn

/tmp/ipykernel_5519/4113099481.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  avg_burn['total_burn'].fillna(0) / avg_burn['number_retailers'],


,campaign_id,retailer_tag,number_retailers,active_post,active_pre,converted,nmv_post,nmv_pre,total_burn,nmv_redeemers,pct_active_post,pct_active_pre,active_delta,nmv_delta,pct_active_delta,AVERAGE_BURN
0,34,active,8,0,0,0,0.0,0.0,None,0.0,0.000000,0.000000,0,0.0,0.000000,0.0


In [95]:
avg_burn = avg_burn[['retailer_tag', 'AVERAGE_BURN']]
avg_burn

,retailer_tag,AVERAGE_BURN
0,active,0.0


In [96]:
merged_df = new_offers.merge(
    avg_burn,
    how='left',              # or 'inner' if you only want matching rows
    left_on='retailer_type',
    right_on='retailer_tag'
)
merged_df

,retailer_id,retailer_type,supplier_size,disc_formatted,offer_ts,flat_reward,max_redeem_rate,avg_redeem_rate,sending_cost,max_exp_burn,avg_exp_burn,max_burn,total_cost,fake_rk,retailer_tag,AVERAGE_BURN
0,669974,active,large,"[33,1,400,50000]",400.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,1,active,0.0
1,114999,active,large,"[205,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,2,active,0.0
2,20102,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,3,active,0.0
3,179817,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,4,active,0.0
4,319941,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,5,active,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,213854,active,large,"[33,1,2100,50000]",2100.0,42.0,0.5,0.30,2,21.0,12.6,42.0,23.0,86,active,0.0
86,378211,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,87,active,0.0
87,536939,active,large,"[33,1,1700,50000]",1700.0,34.0,0.5,0.30,2,17.0,10.2,34.0,19.0,88,active,0.0
88,26641,active,large,"[205,1,1100,50000]",1100.0,22.0,0.5,0.30,2,11.0,6.6,22.0,13.0,89,active,0.0


In [98]:
merged_df['running_cost'] = merged_df['AVERAGE_BURN'].cumsum()
merged_df

,retailer_id,retailer_type,supplier_size,disc_formatted,offer_ts,flat_reward,max_redeem_rate,avg_redeem_rate,sending_cost,max_exp_burn,avg_exp_burn,max_burn,total_cost,fake_rk,retailer_tag,AVERAGE_BURN,running_cost
0,669974,active,large,"[33,1,400,50000]",400.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,1,active,0.0,0.0
1,114999,active,large,"[205,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,2,active,0.0,0.0
2,20102,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,3,active,0.0,0.0
3,179817,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,4,active,0.0,0.0
4,319941,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,5,active,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,213854,active,large,"[33,1,2100,50000]",2100.0,42.0,0.5,0.30,2,21.0,12.6,42.0,23.0,86,active,0.0,0.0
86,378211,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,87,active,0.0,0.0
87,536939,active,large,"[33,1,1700,50000]",1700.0,34.0,0.5,0.30,2,17.0,10.2,34.0,19.0,88,active,0.0,0.0
88,26641,active,large,"[205,1,1100,50000]",1100.0,22.0,0.5,0.30,2,11.0,6.6,22.0,13.0,89,active,0.0,0.0


In [99]:
filtered_df = merged_df[merged_df['running_cost'] < budget_left]
filtered_df


,retailer_id,retailer_type,supplier_size,disc_formatted,offer_ts,flat_reward,max_redeem_rate,avg_redeem_rate,sending_cost,max_exp_burn,avg_exp_burn,max_burn,total_cost,fake_rk,retailer_tag,AVERAGE_BURN,running_cost
0,669974,active,large,"[33,1,400,50000]",400.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,1,active,0.0,0.0
1,114999,active,large,"[205,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,2,active,0.0,0.0
2,20102,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,3,active,0.0,0.0
3,179817,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,4,active,0.0,0.0
4,319941,active,large,"[33,1,500,50000]",500.0,10.0,0.5,0.30,2,5.0,3.0,10.0,7.0,5,active,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,213854,active,large,"[33,1,2100,50000]",2100.0,42.0,0.5,0.30,2,21.0,12.6,42.0,23.0,86,active,0.0,0.0
86,378211,active,large,"[33,1,600,50000]",600.0,12.0,0.5,0.30,2,6.0,3.6,12.0,8.0,87,active,0.0,0.0
87,536939,active,large,"[33,1,1700,50000]",1700.0,34.0,0.5,0.30,2,17.0,10.2,34.0,19.0,88,active,0.0,0.0
88,26641,active,large,"[205,1,1100,50000]",1100.0,22.0,0.5,0.30,2,11.0,6.6,22.0,13.0,89,active,0.0,0.0


In [117]:
import datetime
now = datetime.datetime.now()
from_time = (now + datetime.timedelta(hours=5)).strftime("%d-%m-%Y %H:%M")
to_time = (now + datetime.timedelta(days=3)).strftime("%d-%m-%Y %H:%M")

# Mapping logic for retailer_type -> suffix
suffix_map = {
    "active": "act",
    "active-competition": "act_c",
    "churned-old": "ch_o",
    "churned-recent": "ch_r"
}

# Base campaign/supplier prefix
base = f"ct_{campaign_id_value}_sup{supplier_id_value}"

# --- Build the list of offer dictionaries ---
offers = []

for i, row in filtered_df.iterrows():
    retailer_type = str(row.get("retailer_type", "")).lower()
    short_suffix = suffix_map.get(retailer_type, retailer_type)
    english_tag = f"{base}_{short_suffix}"

    offer = {
        "name": f"Offer tool: {supplier_id_value}",
        "description": english_tag,
        "activation": "1",
        "discount_value": row.get("flat_reward", 0),
        "from": from_time,
        "to": to_time,
        "priority": "1",
        "skus_count": "0",
        "type(0 => percentage , 1 => amount )": "1",
        "sales_orders_ticket_size": row.get("offer_ts", 0),
        "min_number_of_orders": "0",
        "max_number_of_orders": "10000",
        "area": "all",
        "channels": "retailer",
        "payment_method": "all",
        "max_time_per_retailer": "1",
        "total_time_usage": "10000",
        "categories_count": "0",
        "categories_included[id , min_skus , min , max]": "",
        "brands_included[id , min_skus , min , max]": "",
        "category_excluded": "",
        "brand_excluded": "",
        "days_between_orders": "0",
        "union_brands_included[id , min_skus , min , max]": (f"[{row['disc_formatted']}]" if pd.notna(row.get('disc_formatted')) else '[[]]'),
        "union_categories_included[id , min_skus , min , max]": "",
        "white_list": row.get("retailer_id", ""),
        "black_list": "",
        "skus_included[id , min , max]": "",
        "union_skus_included[id , min , max]": "",
        "is_bundle": "0",
        "over_above": "0",
        "sku_group_min_count": "",
        "sku_group_ids": "",
        "group_name": "",
        "sku_group_min_gmv": "",
        "tag": "",
        "french_group_name": "",
        "bundle_group_min_gmv": "",
        "bundle_group_min_count": "",
        "bundle_group_ids": "",
        "english_group_name": "",
        "swahili_group_name": "",
        "rwandan_group_name": ""
    }

    offers.append(offer)

In [118]:
offers_df = pd.DataFrame(offers)
offers_df

,name,description,activation,discount_value,from,to,priority,skus_count,"type(0 => percentage , 1 => amount )",sales_orders_ticket_size,...,group_name,sku_group_min_gmv,tag,french_group_name,bundle_group_min_gmv,bundle_group_min_count,bundle_group_ids,english_group_name,swahili_group_name,rwandan_group_name
0,Offer tool: 36,ct_34_sup36_act,1,10.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,400.0,...,,,,,,,,,,
1,Offer tool: 36,ct_34_sup36_act,1,10.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,500.0,...,,,,,,,,,,
2,Offer tool: 36,ct_34_sup36_act,1,12.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,600.0,...,,,,,,,,,,
3,Offer tool: 36,ct_34_sup36_act,1,10.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,500.0,...,,,,,,,,,,
4,Offer tool: 36,ct_34_sup36_act,1,10.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,500.0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Offer tool: 36,ct_34_sup36_act,1,42.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,2100.0,...,,,,,,,,,,
86,Offer tool: 36,ct_34_sup36_act,1,12.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,600.0,...,,,,,,,,,,
87,Offer tool: 36,ct_34_sup36_act,1,34.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,1700.0,...,,,,,,,,,,
88,Offer tool: 36,ct_34_sup36_act,1,22.0,21-10-2025 19:23,24-10-2025 14:23,1,0,1,1100.0,...,,,,,,,,,,


In [120]:
offers_df.to_excel("test_data.xlsx", index=False)

In [164]:
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",    
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token
jwt_token = get_jwt()


In [140]:
import requests

url = "https://api.maxab.info/main-system/api/admin-portal/offers/upload-offers"

payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"sheet\"; filename=\"test_data.xlsx\"\r\nContent-Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet\r\n\r\n\r\n-----011000010111000001101001--\r\n"
headers = {
    "authorization": f"Bearer {jwt_token}",
    "Content-Type": "multipart/form-data; boundary=---011000010111000001101001"
}

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

{"success":false,"message":"\u064a\u062c\u0628 \u0623\u0646 \u064a\u0643\u0648\u0646 \u0645\u0644\u0641\u064b\u0627 \u0645\u0646 \u0646\u0648\u0639 : xlsx, csv, xls.","msg":"\u064a\u062c\u0628 \u0623\u0646 \u064a\u0643\u0648\u0646 \u0645\u0644\u0641\u064b\u0627 \u0645\u0646 \u0646\u0648\u0639 : xlsx, csv, xls.","details":{"sheet":["\u064a\u062c\u0628 \u0623\u0646 \u064a\u0643\u0648\u0646 \u0645\u0644\u0641\u064b\u0627 \u0645\u0646 \u0646\u0648\u0639 : xlsx, csv, xls."]}}


In [148]:
def send_offers_excel_to_api(offers_df, jwt_token, campaign_id):
    """
    Convert offers_df to Excel format and send to the API
    """
    import io
    from openpyxl import Workbook
    from openpyxl.utils.dataframe import dataframe_to_rows
    
    # Create Excel file in memory
    wb = Workbook()
    ws = wb.active
    ws.title = "Offers"
    
    # Add data from DataFrame to Excel
    for r in dataframe_to_rows(offers_df, index=False, header=True):
        ws.append(r)
    
    # Save to BytesIO buffer
    excel_buffer = io.BytesIO()
    wb.save(excel_buffer)
    excel_buffer.seek(0)
    
       # Prepare the multipart form data with proper field name
    payload = f"-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"sheet\"; filename=\"excel_buffer.getvalue()\"\r\nContent-Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet\r\n\r\n\r\n-----011000010111000001101001--\r\n"

    
    headers = {
        "authorization": f"Bearer {jwt_token}"
    }
    
    url = "https://api.maxab.info/main-system/api/admin-portal/offers/upload-offers"
    
    # Send the request
    response = requests.post(url, data=payload, headers=headers)
    
    return response

# Send the offers_df as Excel to the API
try:
    response = send_offers_excel_to_api(offers_df, jwt_token, campaign_id_value)
    print(f"API Response Status: {response.status_code}")
    print(f"API Response: {response.text}")
except Exception as e:
    print(f"Error sending offers to API: {str(e)}")

API Response Status: 400
API Response: {"success":false,"message":"sheet \u0645\u0637\u0644\u0648\u0628.","msg":"sheet \u0645\u0637\u0644\u0648\u0628.","details":{"sheet":["sheet \u0645\u0637\u0644\u0648\u0628."]}}


In [165]:
import pandas as pd
import requests

# Save the Excel file locally
offers_df.to_excel("Offers_ID_34.xlsx", index=False, engine="openpyxl")

# API details
url = "https://api.maxab.info/main-system/api/admin-portal/offers/upload-offers"
headers = {
    "authorization": f"Bearer {jwt_token}"
}

# Upload with correct field name
with open("Offers_ID_34.xlsx", "rb") as f:
    files = {
        "sheet": ("Offers_ID_34.xlsx", f, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    }
    response = requests.post(url, headers=headers, files=files)

print(response.status_code)
print(response.text)


Field name: sheet
200 {"message":"Imported Successfully","data":[]}
--------------------------------------------------------------------------------
Field name: file
400 {"success":false,"message":"sheet \u0645\u0637\u0644\u0648\u0628.","msg":"sheet \u0645\u0637\u0644\u0648\u0628.","details":{"sheet":["sheet \u0645\u0637\u0644\u0648\u0628."]}}
--------------------------------------------------------------------------------
Field name: upload
403 {"status":403,"message":"max limit reached"}
--------------------------------------------------------------------------------
Field name: offers
403 {"status":403,"message":"max limit reached"}
--------------------------------------------------------------------------------
